# Imports

In [0]:
import numpy as np
import os
import sys
%pip install -U imgaug

IN_COLAB = 'google.colab' in sys.modules

## Import notebooks

In [2]:
from helpers import *
from CNNv2 import CNN

Using TensorFlow backend.


# load images

In [3]:
# Load the training set
if (IN_COLAB):
    from google.colab import drive
    drive.mount('/gdrive')
    root_dir = "/gdrive/My Drive/ML/training/"
else:
    root_dir = "training/"

image_dir = root_dir + "images/"
files = os.listdir(image_dir)
n = len(files)
print("Loading " + str(n) + " images")
imgs = np.asarray([load_image(image_dir + files[i]) for i in range(n)])

gt_dir = root_dir + "groundtruth/"
print("Loading " + str(n) + " groundtruth")
gt_imgs = np.asarray([load_image(gt_dir + files[i]) for i in range(n)])

Drive already mounted at /gdrive; to attempt to forcibly remount, call drive.mount("/gdrive", force_remount=True).
Loading 100 images
Loading 100 groundtruth


# Load model

In [0]:
model = CNN(rootdir=root_dir, window_size=256, nb_epochs=5, train_batch_size=16)

## Train model

In [0]:
#model.load(os.path.join(root_dir, 'best_003.h5'))
#model.load('saved_weights.h5') # set initial_epoch to resume training
INITIAL_EPOCH = 0

In [0]:
model.train(imgs, gt_imgs, initial_epoch=INITIAL_EPOCH)

246/750 [========>.....................] - ETA: 18:48 - loss: 0.6893 - acc: 0.8153

In [0]:
model.save('saved_weights.h5')